In [11]:
import pyomo.environ as pyo
import pandas as pd
from pathlib import Path
import preprocess_data as dat

known_x_sols = [[8, 14, 17, 19, 29, 33, 35, 38, 51, 55, 59, 68, 75, 78, 87],
              [4, 8, 14, 18, 29, 33, 35, 38, 51, 55, 59, 68, 75, 78, 87],
              [8, 14, 17, 18, 29, 33, 35, 38, 51, 55, 59, 68, 75, 78, 87]]

## Data

In [12]:
data_path = Path("../data")
year = 2021

adjacent_list = dat.create_adjacent_list(data_path)
adjacent_matrix = dat.gen_adjacent_matrix(adjacent_list)
df_init, counties = dat.data_ingest(data_path, year)
df = dat.create_df(df_init, counties, adjacent_list)
model = pyo.AbstractModel()

## Defs

In [13]:
def param_adjacent(m, i, j):
    return int(j in adjacent_matrix[i])

def con_a(m, i):
    return sum((m.a[i, j] * m.x[j]) for j in m.J) >= 1

def obj_sum(m):
    return pyo.summation(m.x)

## Constraints

In [14]:
# value of n (number of counties)
model.n = 88

# range of i and j (iterating over counties)
model.I = pyo.RangeSet(1, model.n)
model.J = pyo.RangeSet(1, model.n)

# model.a = pyo.Set(model.I, model.J, within=pyo.Binary, initialize=param_adjacent)  # 1 if county i and j are adjacent
model.a = pyo.Param(model.I, model.J, domain=pyo.Binary, initialize=param_adjacent)  # 1 if county i and j are adjacent

model.x = pyo.Var(model.J, domain=pyo.Binary)  # 1 if principal place of business is opened in county j

model.obj = pyo.Objective(rule=obj_sum, sense=pyo.minimize)

model.a_constraint = pyo.Constraint(model.I, rule=con_a)

## Solve

In [15]:
instance = model.create_instance()
# opt = pyo.SolverFactory("ipopt", executable="/home/adb/anaconda3/bin/ipopt")
opt = pyo.SolverFactory("glpk", executable="/home/adb/anaconda3/bin/glpsol")
# opt = pyo.SolverFactory("glpk", executable="/usr/local/Caskroom/miniconda/base/bin/glpsol")
results = opt.solve(instance).write()
results
# instance.pprint()
# instance.solutions.store_to(results)
# model21.pprint()
# model21.x.get_values()
# solution.print_data()
# solution.solutions.store_to(results)

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 16.0
  Upper bound: 16.0
  Number of objectives: 1
  Number of constraints: 89
  Number of variables: 89
  Number of nonzeros: 542
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 7
      Number of created subproblems: 7
  Error rc: 0
  Time: 0.006479501724243164
# ----------------------------------------------------------
#   Solution Information
# ----------------------------------

In [16]:
sol_dict = instance.x.get_values()
sol1 = [k for k, v in sol_dict.items() if v == 1]
sol1


[3, 5, 7, 8, 9, 28, 35, 40, 45, 49, 51, 69, 72, 75, 76, 81]

In [17]:
len(sol1), len(known_x_sols[0])

15

In [19]:
instance.obj.pprint()

obj : Size=1, Index=None, Active=True
    Key  : Active : Sense    : Expression
    None :   True : minimize : x[1] + x[2] + x[3] + x[4] + x[5] + x[6] + x[7] + x[8] + x[9] + x[10] + x[11] + x[12] + x[13] + x[14] + x[15] + x[16] + x[17] + x[18] + x[19] + x[20] + x[21] + x[22] + x[23] + x[24] + x[25] + x[26] + x[27] + x[28] + x[29] + x[30] + x[31] + x[32] + x[33] + x[34] + x[35] + x[36] + x[37] + x[38] + x[39] + x[40] + x[41] + x[42] + x[43] + x[44] + x[45] + x[46] + x[47] + x[48] + x[49] + x[50] + x[51] + x[52] + x[53] + x[54] + x[55] + x[56] + x[57] + x[58] + x[59] + x[60] + x[61] + x[62] + x[63] + x[64] + x[65] + x[66] + x[67] + x[68] + x[69] + x[70] + x[71] + x[72] + x[73] + x[74] + x[75] + x[76] + x[77] + x[78] + x[79] + x[80] + x[81] + x[82] + x[83] + x[84] + x[85] + x[86] + x[87] + x[88]


In [20]:
instance.a_constraint.pprint()

a_constraint : Size=88, Index=I, Active=True
    Key : Lower : Body                                                                  : Upper : Active
      1 :   1.0 :                                   x[1] + x[8] + x[36] + x[66] + x[73] :  +Inf :   True
      2 :   1.0 :                           x[2] + x[6] + x[32] + x[33] + x[69] + x[81] :  +Inf :   True
      3 :   1.0 :          x[3] + x[38] + x[39] + x[42] + x[47] + x[52] + x[70] + x[85] :  +Inf :   True
      4 :   1.0 :                                          x[4] + x[28] + x[43] + x[78] :  +Inf :   True
      5 :   1.0 :                  x[5] + x[37] + x[53] + x[58] + x[64] + x[82] + x[84] :  +Inf :   True
      6 :   1.0 :           x[2] + x[6] + x[19] + x[33] + x[46] + x[54] + x[75] + x[81] :  +Inf :   True
      7 :   1.0 :                          x[7] + x[30] + x[34] + x[41] + x[56] + x[61] :  +Inf :   True
      8 :   1.0 :                                   x[1] + x[8] + x[13] + x[14] + x[36] :  +Inf :   True
      9 : 

In [ ]:
instance.obj.pprint()